In [ ]:
import json
import time
import calendar
import getpass
import requests
import numpy as np
import geopandas as gpd
import zipfile
import io
import matplotlib.pyplot as plt
from pprint import pprint
from osgeo import gdal
from shapely.geometry import box
from pathlib import Path

# Login to NASA Earthdata

In [ ]:
nasa_earthdata_api = 'https://appeears.earthdatacloud.nasa.gov/api/'

nasa_earthdata_user = getpass.getpass(prompt = 'Enter NASA Earthdata Login Username: ')      
nasa_earthdata_password = getpass.getpass(prompt = 'Enter NASA Earthdata Login Password: ') 

token_response = requests.post(f"{nasa_earthdata_api}login", auth=(nasa_earthdata_user, nasa_earthdata_password)).json()

token = token_response['token']                      
head = {'Authorization': f"Bearer {token}"}

del nasa_earthdata_user, nasa_earthdata_password

# Explore Products

In [ ]:
product_response = requests.get(f"{nasa_earthdata_api}product").json()                         
print(f"AppEEARS currently supports {len(product_response)} products.") 
products = {p['ProductAndVersion']: p for p in product_response}

In [ ]:
products

# Show All Products with Description Containing Keyword

In [ ]:
def get_products_with_description(keyword):
    for p in {p['ProductAndVersion'] for p in product_response}:                                            
        if keyword in products[p]['Description']:
            pprint(products[p])

## Vegetation
Below is all products with a description that contains the keyword "Vegetation"

In [ ]:
get_products_with_description(keyword='Vegetation')

## Elevation
Below is all products with a description that contains the keyword "Elevation"

In [ ]:
get_products_with_description(keyword='Elevation')

# Weather

In [ ]:
get_products_with_description(keyword='Weather')

# Select Products

In [ ]:
selected_products = [
    products['MOD13A3.061'], 
    products['MOD15A2H.061'], 
    products['MOD11A2.061'],
    products['MOD14A2.061'],
    products['MCD64A1.061'],
    products['MOD16A2GF.061'],
    products['DAYMET.004'],
    products['NASADEM_NC.001'],
    products['ASTWBD_ATTNC.001'],
]
print("Selected products: ")
pprint(selected_products)

# Explore Product Layers

In [ ]:
for selected_product in selected_products:
    selected_product_layers = requests.get(f"{nasa_earthdata_api}product/{selected_product['ProductAndVersion']}").json()
    print(f"{selected_product['ProductAndVersion']} Layer ")
    pprint(list(selected_product_layers.keys()))

# Add Layers of Interest

In [ ]:
layers = [
    ('MOD13A3.061', '_1_km_monthly_NDVI'),
    ('MOD13A3.061', '_1_km_monthly_EVI'), 
    ('MOD15A2H.061', 'Lai_500m'),
    ('MOD11A2.061', 'LST_Day_1km'),
    ('MOD14A2.061', 'FireMask'),
    ('MCD64A1.061', 'Burn_Date'),
    ('MCD64A1.061', 'First_Day'),
    ('MCD64A1.061', 'Last_Day'),
    ('MOD16A2GF.061', 'ET_500m'),
    ('DAYMET.004', 'dayl'),
    ('DAYMET.004', 'prcp'),
    ('DAYMET.004', 'tmax'),
    ('DAYMET.004', 'tmin'),
    ('DAYMET.004', 'vp'),
    ('NASADEM_NC.001', 'NASADEM_HGT'),
    ('ASTWBD_ATTNC.001', 'ASTWBD_att')
]
products_layers = [{'product': l[0], 'layer': l[1]} for l in layers]
products_layers

# Explore Available Projections

In [ ]:
projections = requests.get(f"{nasa_earthdata_api}spatial/proj").json()
pprint(projections)
projs = {}                                  
for p in projections: 
    projs[p['Name']] = p 
pprint(list(projs.keys()))

# Define Area of Interest

In [ ]:
canada_boundary_shapefile_path = 'data/canada_boundary/lpr_000b16a_e.shp'

canada_boundary = gpd.read_file(canada_boundary_shapefile_path)
canada_boundary

In [ ]:
print(f"Number of coordinates : {canada_boundary.count_coordinates().sum():,}")
ax = canada_boundary.plot()
ax.set_title(f"Canada Boundary (EPSG:{canada_boundary.crs.to_epsg()})")

## Remove Area Not of Interest

In [ ]:
canada_boundary = canada_boundary[(canada_boundary['PRUID'] != '60') & (canada_boundary['PRUID'] != '61') & (canada_boundary['PRUID'] != '62')]
print(f"Number of coordinates : {canada_boundary.count_coordinates().sum():,}")
ax = canada_boundary.plot()
ax.set_title(f"Canada Boundary (Restricted to South of 60deg) (EPSG:{canada_boundary.crs.to_epsg()})")

# Grid Area of Interest

## Canada-wise Grid

In [ ]:
minx, miny, maxx, maxy = canada_boundary.total_bounds

grid_size_in_meters = 256_000.0 # 500m resolution * 512 pixels = 256 000 meters per side

x_coords = np.arange(minx, maxx, grid_size_in_meters)
y_coords = np.arange(miny, maxy, grid_size_in_meters)
grid_cells = [box(x, y, x + grid_size_in_meters, y + grid_size_in_meters) for x in x_coords for y in y_coords]

grid = gpd.GeoDataFrame({'geometry': grid_cells}, crs=canada_boundary.crs)

grid_within_boundary = grid[grid.intersects(canada_boundary.union_all())]

fig, ax = plt.subplots(figsize=(10, 10))
grid_within_boundary.boundary.plot(ax=ax, edgecolor='black', facecolor='none')
canada_boundary.boundary.plot(ax=ax, edgecolor='red')
ax.set_title(f"Grid Tiles that Intersect with Canada ({len(grid_within_boundary)} tiles)")
plt.show()

# Reproject Area of Interest

In [ ]:
target_epsg = int(projs['geographic']['EPSG'])
canada_boundary = canada_boundary.to_crs(epsg=target_epsg)
grid_within_boundary = grid_within_boundary.to_crs(epsg=target_epsg)

fig, ax = plt.subplots(figsize=(10, 10))
grid_within_boundary.boundary.plot(ax=ax, edgecolor='black', facecolor='none')
ax = canada_boundary.plot(ax=ax, edgecolor='red')
ax.set_title(f"Reprojected Canada & Tiles (EPSG:{canada_boundary.crs.to_epsg()})")
plt.show()

# Submit Task

In [ ]:
def submit_task(
    grid_within_boundary: gpd.GeoDataFrame, 
    start_date_mm_dd_yyyy: str, 
    end_date_mm_dd_yyyy: str,
    product_layer: dict,
    task_name: str
) -> dict:
    geo_area_json = json.loads(grid_within_boundary.to_json())
    task_type = "area" # 'area', 'point'
    output_format = 'netcdf4' # 'netcdf4', 'geotiff'
    output_projection = projs['geographic']['Name']
    recurring = False    
    task = {
        "task_type": task_type,
        "task_name": task_name,
        "params": {
            "dates": [
                {
                    "startDate": start_date_mm_dd_yyyy,
                    "endDate": end_date_mm_dd_yyyy,
                    "recurring": recurring
                }
            ],
            "layers": [product_layer],
            'output': {
                'format': {
                    'type': output_format
                },
                'projection': output_projection
            },
            "geo": geo_area_json,
        }
    }

    task_response = requests.post(f"{nasa_earthdata_api}task", json=task, headers=head)
    pprint(f"Status Code: {task_response.status_code} {task_response.reason}")
    task_response_json = task_response.json()
    pprint(f"Task Response JSON: {task_response_json}")
    
    return task_response_json

In [ ]:
timestamp = time.strftime("%Y%m%d-%H%M%S")
year_product_layer_requests = {}
static_products_layers_downloaded = []
for year in range(2001, 2024):
    print(f"==================== Year: {year} ====================")
    for month in range(1, 13):
        print(f"==================== Month: {month:02} ====================")
        _, last_day = calendar.monthrange(year, month)
        for product_layer in products_layers:
            print(f"==================== Product: {product_layer['product']} Layer: {product_layer['layer']} =================")
            
            task_name = f"canada_500m_512pixels_{product_layer['product']}_{product_layer['layer']}_{year}"
            static_product_layer_name = f"{product_layer['product']}_{product_layer['layer']}"
            
            if products[product_layer['product']]['TemporalGranularity'].lower() == 'static' and \
                static_product_layer_name in static_products_layers_downloaded:
                    continue
            
            if products[product_layer['product']]['TemporalGranularity'].lower() == 'static':
                task_name = f"canada_500m_512pixels_{product_layer['product']}_{product_layer['layer']}"
            
            task_response_json = submit_task(
                grid_within_boundary=grid_within_boundary, 
                start_date_mm_dd_yyyy=f"{month:02}-01-{year}", 
                end_date_mm_dd_yyyy=f"{month:02}-{last_day}-{year}",
                product_layer=product_layer,
                task_name=f"canada_500m_512pixels_{product_layer['product']}_{product_layer['layer']}_{year}"
            )
            
            if products[product_layer['product']]['TemporalGranularity'].lower() == 'static':
                static_products_layers_downloaded.append(static_product_layer_name)
            
            year_product_layer_requests[(f"{year}_{product_layer['product']}_{product_layer['layer']}")] = task_response_json['task_id']
            
            with open(f"data/year_product_layer_requests_{timestamp}.json", 'w') as f:
                json.dump(year_product_layer_requests, f)

# Download Data
You must wait until the data is processed.  
You should receive an email when it is done or you can check via https://appeears.earthdatacloud.nasa.gov/explore

In [ ]:
with open('data/year_product_layer_requests.json', 'r') as f:
    year_product_layer_requests = json.load(f)

In [ ]:
task_id = '796ea315-54c1-4801-bef3-7bb80bff202f' #  TODO : Use year_product_layer_requests
bundle = requests.get(f"{nasa_earthdata_api}bundle/{task_id}", headers=head).json()
bundle 

In [ ]:
files = {}                                                      
for file_id in bundle['files']: 
    files[file_id['file_id']] = file_id['file_name']  
files

In [ ]:
features_dir = Path('data/features/')

In [ ]:
for file_id in files.keys():
    if not files[file_id].endswith('.nc') or "NUMNC" in files[file_id]:
        continue
    dl = requests.get(f"{nasa_earthdata_api}bundle/{task_id}/{file_id}", headers=head, stream=True, allow_redirects = 'True')                                # Get a stream to the bundle file
    if files[file_id].endswith('.tif'):
        filename = files[file_id].split('/')[1]
    else:
        filename = files[file_id] 
    with open(features_dir / Path(filename), 'wb') as file_id:                                                                  
        for data in dl.iter_content(chunk_size=8192): 
            file_id.write(data) 
print(f"Downloaded files can be found at: {features_dir}")

# Preview Features

In [ ]:
features_ds = gdal.Open(str((features_dir / 'MOD13Q1.061_250m_aid0001.nc').resolve()))
features_proj = features_ds.GetProjection()

for subdataset in features_ds.GetSubDatasets():
    band_ds = gdal.Open(subdataset[0])
    band = band_ds.GetRasterBand(1)
    band_type = gdal.GetDataTypeName(band.DataType)
    band_min, band_max, band_mean, band_stddev = band.GetStatistics(True, True)
    band_desc = band_ds.GetDescription().split(':')[-1]
    print(f"Band {band_desc}: ({band.GetYSize()},{band.GetXSize()}) Type={band_type}, Min={band_min}, Max={band_max}, Mean={band_mean}, StdDev={band_stddev}")
    print(f"Band Projection : {band_ds.GetProjection()}")
    print("")
    

    band_data = band_ds.ReadAsArray()

    plt.figure(figsize=(8, 6))
    plt.imshow(band_data.transpose((1, 2, 0)), cmap='RdYlGn')
    plt.colorbar(label=band_desc)
    plt.title(band_desc)
    plt.xlabel('Width')
    plt.ylabel('Height')
    plt.show()

# Download Target

In [ ]:
target_dir = Path('data/target/')

In [ ]:
nbac_response = requests.get("https://cwfis.cfs.nrcan.gc.ca/downloads/nbac/nbac_1972_2023_20240530_shp.zip")
zipfile.ZipFile(io.BytesIO(nbac_response.content)).extractall(target_dir)

In [ ]:
target_shp = list(target_dir.glob('*.shp'))[0]
target_shp

# Preview Target

In [ ]:
gdf = gpd.read_file(target_shp, rows=1)

fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(ax=ax, edgecolor='k', facecolor='none')
plt.title('Subset of Target')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

# Reproject Target

In [ ]:
reprojected_target_shp = Path(target_shp.parent, f"{target_shp.stem}_reprojected.shp")

In [ ]:
gdal.VectorTranslate(
    str(reprojected_target_shp.resolve()), 
    str(target_shp.resolve()), 
    dstSRS='EPSG:4326',
    reproject=True
)

# Preview Target

In [ ]:
gdf = gpd.read_file(reprojected_target_shp, rows=1)

fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(ax=ax, edgecolor='k', facecolor='none')
plt.title('Subset of Reprojected Target')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()